In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import os

In [ ]:
EPOCH = 10
LR_RATE = 0.01

In [ ]:
# vgg16网络结构，输入图像大小变化： 32-> 16->  8->  4->  2->  1
# vgg16网络结构，输入通道大小变化：  3-> 64->128->256->512->512
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            #1
            nn.Conv2d(3,64,kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            #2
            nn.Conv2d(64,64,kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            #3
            nn.Conv2d(64,128,kernel_size=3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            #4
            nn.Conv2d(128,128,kernel_size=3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            #5
            nn.Conv2d(128,256,kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            #6
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            #7
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            #8
            nn.Conv2d(256,512,kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            #9
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            #10
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            #11
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            #12
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            #13
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.AvgPool2d(kernel_size=1,stride=1),
            )
        self.classifier = nn.Sequential(
            #14
            nn.Linear(512,4096),
            nn.ReLU(True),
            nn.Dropout(),
            #15
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            #16
            nn.Linear(4096,num_classes),
            )
        #self.classifier = nn.Linear(512, 10)
 
    def forward(self, x):
        out = self.features(x) 
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), #先四周填充0，再把图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])
 
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=0)
 
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=0)
#Cifar-10的标签
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(trainset.__len__())
print(len(trainloader))

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# 模型定义 VGG16
net = VGG16().to(device)
print(net)

In [ ]:
# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()   # 损失函数使用交叉熵
# CrossEntropyLoss会计算出每一个样本的交叉熵并在batchsize上取平均，返回的是一个标量数值
optimizer = optim.SGD(net.parameters(), lr=LR_RATE, momentum=0.9, weight_decay=5e-4) # 优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

In [ ]:
for epoch in range(0, EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    #开始训练
    net.train()
    #总损失
    sum_loss = 0.0
    #准确率
    accuracy = 0.0
    total = 0.0

    for i, data in enumerate(trainloader):
        #准备数据
        length = len(trainloader)           # 数据大小
        inputs, labels = data               # 取出数据
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()               # 梯度初始化为零（因为一个batch的loss关于weight的导数是所有sample的loss关于weight的导数的累加和）
        inputs, labels = Variable(inputs), Variable(labels)
        # forward + backward + optimize
        outputs = net(inputs)               # 前向传播求出预测值
        loss = criterion(outputs, labels)   # 求loss
        loss.backward()                     # 反向传播求梯度
        optimizer.step()                    # 更新参数

        # 每一个batch输出对应的损失loss和准确率accuracy
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)           # 返回每一行中最大值的那个元素，且返回其索引
        # predicted为batchsize*1的tensor，其中对应值为对应输入的预测结果
        total += labels.size(0)                             # 记录一共使用了多少个样本
        accuracy += predicted.eq(labels.data).cpu().sum()   # 预测值和真实值进行比较，将数据放到cpu上并且求和

        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
                         % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * accuracy / total))


In [ ]:
PATH = './train_ipynb.pth'
torch.save(net.state_dict(),PATH)

In [ ]:
accuracy = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        accuracy += predicted.eq(labels.data).cpu().sum()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (
    100 * accuracy / total))